# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders=pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
#1

uni=orders.groupby('CustomerID').agg('sum')
u=uni['amount_spent'].sort_values(ascending=False).reset_index()
display(u.head())

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06


In [4]:
#2

p_25=u['amount_spent'].quantile(0.25)
p_75=u['amount_spent'].quantile(0.75)

IQR=p_75-p_25
print (IQR)

1354.3950000000004


In [5]:
rango_IQR=u[(u['amount_spent']>=p_25)&(u['amount_spent']<p_75)]
display(rango_IQR.head())

,CustomerID,amount_spent
1085,15214,1661.44
1086,15024,1661.33
1087,15332,1661.06
1088,12967,1660.90
1089,14045,1659.75


In [9]:
#3
p_95=u['amount_spent'].quantile(0.95)

pref=u[(u['amount_spent']>=p_75)&(u['amount_spent']<p_95)]
vip=u[(u['amount_spent']>=p_95)]


def customer(x):
    if p_75 <= x < p_95:
        return 'Pref'
    elif x>=p_95:
        return 'Vip'
    else:
        return 'Normal'
    
    
u['Class']=u['amount_spent'].apply(customer)
u.head()

,CustomerID,amount_spent,Class
0,14646,280206.02,Vip
1,18102,259657.30,Vip
2,17450,194550.79,Vip
3,16446,168472.50,Vip
4,14911,143825.06,Vip


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [12]:
# your code here
merg=u.merge(orders[['CustomerID','Country']], how='left', on='CustomerID').drop_duplicates()
merg.head()

,CustomerID,amount_spent,Class,Country
0,14646,280206.02,Vip,Netherlands
2080,18102,259657.30,Vip,United Kingdom
2511,17450,194550.79,Vip,United Kingdom
2848,16446,168472.50,Vip,United Kingdom
2851,14911,143825.06,Vip,EIRE


In [40]:
vip=merg[merg['Class']=='Vip'].groupby('Country').agg('count')['CustomerID'].sort_values(ascending=False)
display(pd.DataFrame(vip).reset_index().head())

,Country,CustomerID
0,United Kingdom,177
1,Germany,10
2,France,9
3,Switzerland,3
4,Spain,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [32]:
# your code here
pref=merg[merg['Class']=='Pref'].groupby('Country').agg('count')['CustomerID'].sort_values(ascending=False)
pref=pd.DataFrame(pref).reset_index()

In [39]:
v_p=pd.DataFrame(vip).reset_index().merge(pref, how='left', on='Country')
v_p['Total']=v_p.CustomerID_x+v_p.CustomerID_y
v_p.sort_values(by='Total', ascending=False).head()

,Country,CustomerID_x,CustomerID_y,Total
0,United Kingdom,177,755.0,932.0
1,Germany,10,29.0,39.0
2,France,9,20.0,29.0
17,Belgium,1,11.0,12.0
3,Switzerland,3,6.0,9.0
